In [1]:
! pip install yt-dlp


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from yt_dlp import YoutubeDL

https://github.com/ytdl-org/youtube-dl?tab=readme-ov-file#options

In [11]:
#options
import os
parent_dir = os.path.dirname(os.getcwd())
print(f'Parent directory: {parent_dir}')
print(os.getcwd())
download_dir = os.path.join(parent_dir, 'downloads')
os.makedirs(download_dir, exist_ok=True)

#check if ffmpeg exists, if not, download it
import shutil
ffmpeg_dir = shutil.which('ffmpeg')
if ffmpeg_dir:
    print(f'FFmpeg found at {ffmpeg_dir}.')
    ffmpeg_dir = os.path.dirname(ffmpeg_dir)
else:
    print('FFmpeg not found.')
    ffmpeg_dir = os.path.join(parent_dir, 'ffmpeg')

Parent directory: c:\Users\Hero\Documents\GitHub\Youtube-dl-User-Interface
c:\Users\Hero\Documents\GitHub\Youtube-dl-User-Interface\yt_dl_gui
FFmpeg not found.


### Downloading FFmpeg

In [4]:
def download_ffmpeg() -> None:
    import requests
    import zipfile
    import io
    import time
    import sys

    # URL of FFmpeg Essentials Build (Windows 64-bit)
    url = "https://www.gyan.dev/ffmpeg/builds/ffmpeg-release-essentials.zip"
    zip_path = os.path.join(ffmpeg_dir, 'ffmpeg.zip')

    print('Downloading FFmpeg...')
    response = requests.get(url=url, stream=True)
    downloaded = 0
    start_time = time.time()

    with open(zip_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
            downloaded += len(chunk)
            elapsed_time = time.time() - start_time
            speed = downloaded / (1024*1024) / elapsed_time #MB/s
            sys.stdout.flush()
            sys.stdout.write('\rDownloading at {0:.2f} MB/s'.format(speed).ljust(30))

    #extract only ffmpeg.exe and ffprobe.exe
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        for file in zip_ref.namelist():
            if file.endswith('ffmpeg.exe') or file.endswith('ffprobe.exe'):
                # Extract to ffmpeg_dir, flattening the directory structure
                target_path = os.path.join(ffmpeg_dir, os.path.basename(file))
                with zip_ref.open(file) as source, open(target_path, 'wb') as target:
                    target.write(source.read())
    
    os.remove(zip_path)

In [5]:
if not (os.path.exists(os.path.join(ffmpeg_dir, 'ffmpeg.exe')) or os.path.exists(os.path.join(ffmpeg_dir, 'ffprobe.exe'))):
    download_ffmpeg()

### MP4

In [6]:
ydl_options = {
    'format': 'best',
    'outtmpl': os.path.join(download_dir, '%(title)s.%(ext)s'),
}

### FLAC

In [7]:
ydl_options = {
    'format': 'bestaudio/best',
    'outtmpl': os.path.join(download_dir, '%(title)s.%(ext)s'),
    'ffmpeg_location': ffmpeg_dir,
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'flac',
    }]
}

### MP3  
(note: mp3 typically delays signal by 10-50ms)

In [8]:
ydl_options = {
    'format': 'bestaudio/best',
    'outtmpl': os.path.join(download_dir, '%(title)s.%(ext)s'),
    'ffmpeg_location': ffmpeg_dir,
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '0', # tells ffmpeg to use VBR
    }]
}

In [9]:
URL = 'https://www.youtube.com/watch?v=KZt4AVVR_aU'

with YoutubeDL(ydl_options) as ydl:
    ydl.download([URL])

[youtube] Extracting URL: https://www.youtube.com/watch?v=KZt4AVVR_aU
[youtube] KZt4AVVR_aU: Downloading webpage
[youtube] KZt4AVVR_aU: Downloading tv simply player API JSON
[youtube] KZt4AVVR_aU: Downloading tv client config
[youtube] KZt4AVVR_aU: Downloading tv player API JSON
[info] KZt4AVVR_aU: Downloading 1 format(s): 251
[download] Sleeping 3.00 seconds as required by the site...
[download] Destination: c:\Users\Hero\Documents\GitHub\Youtube-dl-User-Interface\downloads\MIKUMV.webm
[download] 100% of  353.10KiB in 00:00:00 at 2.43MiB/s   
[ExtractAudio] Destination: c:\Users\Hero\Documents\GitHub\Youtube-dl-User-Interface\downloads\MIKUMV.mp3
Deleting original file c:\Users\Hero\Documents\GitHub\Youtube-dl-User-Interface\downloads\MIKUMV.webm (pass -k to keep)
